# Feature Selection
In machine learning and statistics, feature selection, also known as variable selection, attribute selection or variable subset selection, is the process of selecting a subset of relevant features (variables, predictors) for use in model construction. Feature selection techniques are used for four reasons:
1. simplification of models to make them easier to interpret by researchers/users
2. shorter training times,
3. to avoid the curse of dimensionality,
4. enhanced generalization by reducing overfitting(formally, reduction of variance)

The central premise when using a feature selection technique is that the data contains some features that are either __redundant or irrelevant__, and can thus be removed without incurring much loss of information.


## Load Data and Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import stats
from scipy.stats.stats import pearsonr
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from scipy import stats
from scipy.special import inv_boxcox
from math import sqrt
from sklearn.metrics import r2_score
import seaborn as sns

In [2]:
#pd.set_option('display.height', 1000)
#pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
train = pd.read_csv("data/train.csv")

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [5]:
#Look for catigorical data
train.nunique()

Id               1460
MSSubClass         15
MSZoning            5
LotFrontage       110
LotArea          1073
Street              2
Alley               2
LotShape            4
LandContour         4
Utilities           2
LotConfig           5
LandSlope           3
Neighborhood       25
Condition1          9
Condition2          8
BldgType            5
HouseStyle          8
OverallQual        10
OverallCond         9
YearBuilt         112
YearRemodAdd       61
RoofStyle           6
RoofMatl            8
Exterior1st        15
Exterior2nd        16
MasVnrType          4
MasVnrArea        327
ExterQual           4
ExterCond           5
Foundation          6
                 ... 
BedroomAbvGr        8
KitchenAbvGr        4
KitchenQual         4
TotRmsAbvGrd       12
Functional          7
Fireplaces          4
FireplaceQu         5
GarageType          6
GarageYrBlt        97
GarageFinish        3
GarageCars          5
GarageArea        441
GarageQual          5
GarageCond          5
PavedDrive

In [6]:
#Describe data
train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,567.240411,1057.429452,1162.626712,346.992466,5.844521,1515.463699,0.425342,0.057534,1.565068,0.382877,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,441.866955,438.705324,386.587738,436.528436,48.623081,525.480383,0.518911,0.238753,0.550916,0.502885,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,0.000000,0.000000,334.000000,0.000000,0.000000,334.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,223.000000,795.750000,882.000000,0.000000,0.000000,1129.500000,0.000000,0.000000,1.000000,0.000000,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,477.500000,991.500000,1087.000000,0.000000,0.000000,1464.000000,0.000000,0.000000,2.000000,0.000000,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,808.000000,1298.250000,1391.250000,728.000000,0.000000,1776.750000,1.000000,0.000000,2.000000,1.000000,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,2336.000000,6110.000000,4692.000000,2065.000000,572.000000,5642.000000,3.000000,2.000000,3.000000,2.000000,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [7]:
train.head(20)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.0,14115,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1.5Fin,5,5,1993,1995,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,Wood,Gd,TA,No,GLQ,732,Unf,0,64,796,GasA,Ex,Y,SBrkr,796,566,0,1362,1,0,1,1,1,1,TA,5,Typ,0,NaN,Attchd,1993.0,Unf,2,480,TA,TA,Y,40,30,0,320,0,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,Gd,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,NaN,10382,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,PosN,Norm,1Fam,2Story,7,6,1973,1973,Gable,CompShg,HdBoard,HdBoard,Stone,240.0,TA,TA,CBlock,Gd,TA,Mn,ALQ,859,BLQ,32,216,1107,GasA,Ex,Y,SBrkr,1107,983,0,2090,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1973.0,RFn,2,484,TA,TA,Y,235,204,228,0,0,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.0,6120,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1.5Fin,7,5,1931,1950,Gable,CompShg,BrkFace,Wd Shng,None,0.0,TA,TA,BrkTil,TA,TA,No,Unf,0,Unf,0,952,952,GasA,Gd,Y,FuseF,1022,752,0,1774,0,0,2,0,2,2,TA,8,Min1,2,TA,Detchd,1931.0,Unf,2,468,Fa,TA,Y,90,0,205,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.0,7420,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,BrkSide,Artery,Artery,2fmCon,1.5Unf,5,6,1939,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,GLQ,851,Unf,0,140,991,Gas

In [8]:
# to make this notebook's output stable across runs
random_state=42
np.random.seed(random_state)

## Split Train & Test data

In [9]:
train_set, test_set = train_test_split(train, test_size=0.3, random_state=random_state)

## Select relevant features


All feature selection techniques requires the non numeric data(categorical variables) to be converted into numberic form, hence convert.

Categorical data are variables that contain label values rather than numeric values.
The number of possible values is often limited to a fixed set.
Categorical variables are often called nominal.

Some examples include:
- A “pet” variable with the values: “dog” and “cat“.
- A “color” variable with the values: “red“, “green” and “blue“.
- A “place” variable with the values: “first”, “second” and “third“.
Each value represents a different category.


Some algorithms can work with categorical data directly.
For example, a decision tree can be learned directly from categorical data with no data transform required (this depends on the specific implementation).

Many machine learning algorithms cannot operate on label data directly. They require all input variables and output variables to be numeric.

In general, this is mostly a constraint of the efficient implementation of machine learning algorithms rather than hard limitations on the algorithms themselves.

This means that categorical data must be converted to a numerical form. If the categorical variable is an output variable, you may also want to convert predictions by the model back into a categorical form in order to present them or use them in some application.

### Convert Categorical to numeric values

This involves following steps:

1. __Integer Encoding__
    - As a first step, each unique category value is assigned an integer value. 
    - For example, “red” is 1, “green” is 2, and “blue” is 3.
    - This is called a label encoding or an integer encoding and is easily reversible.
    - For some variables, this may be enough.
    - The integer values have a natural ordered relationship between each other and machine learning algorithms may be able to understand and harness this relationship.
    - For example, ordinal variables like the “place” example above would be a good example where a label encoding would be sufficient.
    - But this process is not widely used.
2. __One-Hot Encoding__
    - For categorical variables where no such ordinal relationship exists, the integer encoding is not enough.
    - In fact, using this encoding and allowing the model to assume a natural ordering between categories may result in poor performance or unexpected results (predictions halfway between categories).
    - In this case, a one-hot encoding can be applied to the integer representation. This is where the integer encoded variable is removed and a new binary variable is added for each unique integer value.
    - __OneHotEncoder__ cannot process string values directly. If your nominal features are strings, then you need to first map them into integers.
    - __You need two steps:__
        1. Use LabelEncoder to encode your string variables to integers
        2. Then use OneHotEncoder on your integer variables
    - __pandas.get_dummies__ is kind of the opposite. By default, it only converts string columns into one-hot representation, unless columns are specified.

3. __Frequency-based encoding__
    - Using this method, the occurrence frequency is calculated for each class out of the total classes. The corresponding relative frequency is assigned as encoded values for the classes.
4. __Target mean encoding__
    - It can only be used for supervised machine learning problem where there is a target or response attribute. Each class of the categorical input attribute is encoded as a function of the mean of the target.
5. __Binary encoding__
    - The classes are first converted to the numeric format, and then they are converted to their equivalent binary strings. The binary digits are then split into separate columns.
6. __Hash encoding__
    - This method is also popularly known as feature hashing. A hash function is used to map data to a number. This method may map different classes to the same bucket, but is useful when there are hundreds of classes/categories present for an attribute.
    

In [10]:
#Divide the train & test dataset into dependent & independent variable
housing_X_train = train_set.iloc[:,:-1]
housing_y_train = train_set.iloc[:,-1:]

housing_X_test = test_set.iloc[:,:-1]
housing_y_test = test_set.iloc[:,-1:]


#### Label encoding : Convert string to integer

In [11]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

for col in housing_X_train.columns.values:
    # Encoding only categorical variables
    if housing_X_train[col].dtypes=='object':
        print(col)
        #Convert NAN to string NAN
        housing_X_train[col].fillna('NAN', inplace=True)
        housing_X_test[col].fillna('NAN', inplace=True)
        
        # Using whole data to form an exhaustive list of levels
        data=housing_X_train[col].append(housing_X_test[col])
        le.fit(data.values)
        housing_X_train[col]=le.transform(housing_X_train[col])
        housing_X_test[col]=le.transform(housing_X_test[col])
    elif housing_X_train[col].dtypes=='int64' or housing_X_train[col].dtypes=='float64' :
        housing_X_train[col].fillna(0, inplace=True)
        
housing_X_train

MSZoning
Street
Alley
LotShape
LandContour
Utilities
LotConfig
LandSlope
Neighborhood
Condition1
Condition2
BldgType
HouseStyle
RoofStyle
RoofMatl
Exterior1st
Exterior2nd
MasVnrType
ExterQual
ExterCond
Foundation
BsmtQual
BsmtCond
BsmtExposure
BsmtFinType1
BsmtFinType2
Heating
HeatingQC
CentralAir
Electrical
KitchenQual
Functional
FireplaceQu
GarageType
GarageFinish
GarageQual
GarageCond
PavedDrive
PoolQC
Fence
MiscFeature
SaleType
SaleCondition


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
135,136,20,3,80.0,10400,1,1,3,3,0,4,0,14,2,2,0,2,7,6,1970,1970,3,1,9,10,1,288.0,3,4,2,4,4,4,6,0,6,0,1304,1304,1,2,1,5,1682,0,0,1682,0,0,2,0,3,1,3,7,6,1,2,1,1970.0,3,2,530,5,5,2,98,0,0,0,0,0,3,2,1,0,5,2008,8,4
1452,1453,180,4,35.0,3675,1,1,3,3,0,4,0,7,2,2,4,7,5,5,2005,2005,1,1,12,13,1,80.0,3,4,2,2,4,1,2,547,6,0,0,547,1,2,1,5,1072,0,0,1072,1,0,1,0,2,1,3,5,6,0,3,2,2005.0,0,2,525,5,5,2,0,28,0,0,0,0,3,4,1,0,5,2006,8,4
762,763,60,1,72.0,8640,1,1,3,3,0,4,0,21,2,2,0,5,7,5,2009,2009,1,1,12,13,3,0.0,3,4,2,2,4,2,2,24,6,0,732,756,1,0,1,5,764,783,0,1547,0,0,2,1,3,1,2,7,6,0,3,1,2009.0,3,2,614,5,5,2,169,45,0,0,0,0,3,4,1,0,6,2010,2,4
932,933,20,3,84.0,11670,1,1,0,3,0,0,0,21,8,2,0,2,9,5,2006,2006,3,1,12,7,4,302.0,0,4,2,0,1,4,6,0,6,0,1905,1905,1,0,1,5,1905,0,0,1905,0,0,2,0,3,1,0,8,6,1,2,1,2006.0,0,3,788,5,5,2,0,191,0,0,0,0,3,4,1,0,3,2007,8,4
435,436,60,3,43.0,10667,1,1,1,3,0,1,0,5,4,2,0,5,7,6,1996,1996,1,1,12,13,3,0.0,2,4,2,2,4,0,2,385,0,344,70,799,1,0,1,5,827,834,0,1661,1,0,2,1,3,1,2,6,6,1,5,1,1996.0,2,2,550,5,5,2,158,61,0,0,0,0,3,4,1,0,4,2009,5,4
629,630,80,3,82.0,9020,1,1,3,3,0,0,0,12,1,2,0,7,6,5,1964,1964,1,7,9,14,1,259.0,3,4,1,4,4,1,2,624,5,336,288,1248,1,4,1,5,1350,0,0,1350,1,0,1,1,3,1,3,6,6,0,3,1,1964.0,2,2,520,5,5,2,176,0,0,0,0,0,3,0,1,0,6,2008,8,4
1210,1211,60,3,70.0,11218,1,1,3,3,0,4,0,20,2,2,0,5,6,5,1992,1992,1,1,13,14,3,0.0,2,4,2,2,4,4,6,0,6,0,1055,1055,1,0,1,5,1055,790,0,1845,0,0,2,1,3,1,2,8,6,1,5,1,1992.0,2,2,462,5,5,2,635,104,0,0,0,0,3,0,3,400,5,2010,8,4
1118,1119,80,3,85.0,13825,1,1,3,3,0,4,0,19,2,2,0,7,5,6,1958,1987,1,1,8,8,3,0.0,3,2,1,4,4,4,6,0,6,0,533,533,1,4,1,5,1021,580,0,1601,0,1,1,0,3,1,3,6,3,0,3,3,1958.0,2,1,300,5,5,2,280,34,0,0,0,0,3,4,1,0,12,2008,8,4
1084,1085,60,3,0.0,13031,1,1,1,3,0,0,0,8,2,2,0,5,6,5,1995,1996,1,1,6,6,3,0.0,3,4,2,2,4,4,0,592,6,0,99,691,1,2,1,5,691,807,0,1498,0,0,2,1,3,1,3,6,6,1,5,1,1995.0,0,2,409,5,5,2,315,44,0,0,0,0,3,4,1,0,7,2006,8,4
158,159,60,1,100.0,12552,1,1,3,3,0,0,0,21,2,2,0,5,7,5,2004,2005,1,1,12,13,3,0.0,2,4,2,2,4,4,2,222,6,0,769,991,1,0,1,5,991,956,0,1947,0,0,2,1,3,1,2,8,6,1,2,1,2004.0,2,2,678,5,5,2,0,136,0,0,0,0,3,1,1,0,5,2010,8,4


#### One-Hot Encoding : Integer to binary representation
The below rows are categorical and hence needs to be one hot encoded

MSZoning,
Street,
Alley,
LotShape,
LandContour,
Utilities,
LotConfig,
LandSlope,
Neighborhood,
Condition1,
Condition2,
BldgType,
HouseStyle,
RoofStyle,
RoofMatl,
Exterior1st,
Exterior2nd,
MasVnrType,
ExterQual,
ExterCond,
Foundation,
BsmtQual,
BsmtCond,
BsmtExposure,
BsmtFinType1,
BsmtFinType2,
Heating,
HeatingQC,
CentralAir,
Electrical,
KitchenQual,
Functional,
FireplaceQu,
GarageType,
GarageFinish,
GarageQual,
GarageCond,
PavedDrive,
PoolQC,
Fence,
MiscFeature,
SaleType,
SaleCondition


In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

enc=OneHotEncoder(sparse=False)

housing_X_train_1=housing_X_train
housing_X_test_1=housing_X_test

columns=['MSZoning','Street','Alley','LotShape','LandContour','Utilities','LotConfig','LandSlope',
         'Neighborhood','Condition1','Condition2','BldgType','HouseStyle','RoofStyle','RoofMatl',
         'Exterior1st','Exterior2nd','MasVnrType','ExterQual','ExterCond','Foundation','BsmtQual',
         'BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','Heating','HeatingQC','CentralAir',
         'Electrical','KitchenQual','Functional','FireplaceQu','GarageType','GarageFinish','GarageQual',
         'GarageCond','PavedDrive','PoolQC','Fence','MiscFeature','SaleType','SaleCondition']

# Iterating over all the common columns in train and test
for col in columns:
    # creating an exhaustive list of all possible categorical values
    data=housing_X_train[[col]].append(housing_X_test[[col]])
    enc.fit(data)

    # Fitting One Hot Encoding on train data
    temp = enc.transform(housing_X_train[[col]])
    # Changing the encoded features into a data frame with new column names
    temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in data[col].value_counts().index])    
    
    # In side by side concatenation index values should be same
    # Setting the index values similar to the housing_X_train data frame
    temp=temp.set_index(housing_X_train.index.values)

    # adding the new One Hot Encoded varibales to the train data frame
    housing_X_train_1=pd.concat([housing_X_train_1,temp],axis=1)
    housing_X_train_1.drop(col,axis=1,inplace=True)


    # Fitting One Hot Encoding on test data
    temp = enc.transform(housing_X_test[[col]])
    # Changing the encoded features into a data frame with new column names
    temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in data[col].value_counts().index])

    # In side by side concatenation index values should be same
    # Setting the index values similar to the housing_X_test data frame
    temp=temp.set_index(housing_X_test.index.values)

    # adding the new One Hot Encoded varibales to the test data frame
    housing_X_test_1=pd.concat([housing_X_test_1,temp],axis=1)
    housing_X_test_1.drop(col,axis=1,inplace=True)

housing_X_train_1

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_3,MSZoning_4,MSZoning_1,MSZoning_2,MSZoning_0,Street_1,Street_0,Alley_1,Alley_0,Alley_2,LotShape_3,LotShape_0,LotShape_1,LotShape_2,LandContour_3,LandContour_0,LandContour_1,LandContour_2,Utilities_0,Utilities_1,LotConfig_4,LotConfig_0,LotConfig_1,LotConfig_2,LotConfig_3,LandSlope_0,LandSlope_1,LandSlope_2,Neighborhood_12,Neighborhood_5,Neighborhood_17,Neighborhood_7,Neighborhood_21,Neighborhood_8,Neighborhood_16,Neighborhood_19,Neighborhood_14,Neighborhood_20,Neighborhood_3,Neighborhood_6,Neighborhood_11,Neighborhood_15,Neighborhood_23,Neighborhood_9,Neighborhood_4,Neighborhood_18,Neighborhood_22,Neighborhood_0,Neighborhood_10,Neighborhood_2,Neighborhood_24,Neighborhood_13,Neighborhood_1,Condition1_2,Condition1_1,Condition1_0,Condition1_6,Condition1_4,Condition1_5,Condition1_3,Condition1_8,Condition1_7,Condition2_2,Condition2_1,Condition2_7,Condition2_4,Condition2_0,Condition2_6,Condition2_5,Condition2_3,BldgType_0,BldgType_4,BldgType_2,BldgType_3,BldgType_1,HouseStyle_2,HouseStyle_5,HouseStyle_0,HouseStyle_7,HouseStyle_6,HouseStyle_1,HouseStyle_4,HouseStyle_3,RoofStyle_1,RoofStyle_3,RoofStyle_0,RoofStyle_2,RoofStyle_4,RoofStyle_5,RoofMatl_1,RoofMatl_5,RoofMatl_7,RoofMatl_6,RoofMatl_4,RoofMatl_3,RoofMatl_2,RoofMatl_0,Exterior1st_12,Exterior1st_6,Exterior1st_8,Exterior1st_13,Exterior1st_9,Exterior1st_5,Exterior1st_3,Exterior1st_14,Exterior1st_11,Exterior1st_0,Exterior1st_10,Exterior1st_2,Exterior1st_7,Exterior1st_4,Exterior1st_1,Exterior2nd_13,Exterior2nd_8,Exterior2nd_6,Exterior2nd_14,Exterior2nd_10,Exterior2nd_5,Exterior2nd_15,Exterior2nd_12,Exterior2nd_3,Exterior2nd_0,Exterior2nd_7,Exterior2nd_2,Exterior2nd_11,Exterior2nd_1,Exterior2nd_9,Exterior2nd_4,MasVnrType_3,MasVnrType_1,MasVnrType_4,MasVnrType_0,MasVnrType_2,ExterQual_3,ExterQual_2,ExterQual_0,ExterQual_1,ExterCond_4,ExterCond_2,ExterCond_1,ExterCond_0,ExterCond_3,Foundation_2,Foundation_1,Foundation_0,Foundation_3,Foundation_4,Foundation_5,BsmtQual_4,BsmtQual_2,BsmtQual_0,BsmtQual_3,BsmtQual_1,BsmtCond_4,BsmtCond_1,BsmtCond_0,BsmtCond_2,BsmtCond_3,BsmtExposure_4,BsmtExposure_0,BsmtExposure_1,BsmtExposure_2,BsmtExposure_3,BsmtFinType1_6,BsmtFinType1_2,BsmtFinType1_0,BsmtFinType1_1,BsmtFinType1_5,BsmtFinType1_3,BsmtFinType1_4,BsmtFinType2_6,BsmtFinType2_5,BsmtFinType2_3,BsmtFinType2_4,BsmtFinType2_1,BsmtFinType2_0,BsmtFinType2_2,Heating_1,Heating_2,Heating_3,Heating_5,Heating_4,Heating_0,HeatingQC_0,HeatingQC_4,HeatingQC_2,HeatingQC_1,HeatingQC_3,CentralAir_1,CentralAir_0,Electrical_5,Electrical_0,Electrical_1,Electrical_2,Electrical_4,Electrical_3,KitchenQual_3,KitchenQual_2,KitchenQual_0,KitchenQual_1,Functional_6,Functional_3,Functional_2,Functional_4,Functional_0,Functional_1,Functional_5,FireplaceQu_3,FireplaceQu_2,FireplaceQu_5,FireplaceQu_1,FireplaceQu_0,FireplaceQu_4,GarageType_1,GarageType_5,GarageType_3,GarageType_6,GarageType_2,GarageType_4,GarageType_0,GarageFinish_3,GarageFinish_2,GarageFinish_0,GarageFinish_1,GarageQual_5,GarageQual_3,GarageQual_1,GarageQual_2,GarageQual_4,GarageQual_0,GarageCond_5,GarageCond_3,GarageCond_1,GarageCond_2,GarageCond_4,GarageCond_0,PavedDrive_2,PavedDrive_0,PavedDrive_1,PoolQC_3,PoolQC_2,PoolQC_1,PoolQC_0,Fence_4,Fence_2,Fence_0,Fence_1,Fence_3,MiscFeature_1,MiscFeature_3,MiscFeature_2,MiscFeature_0,MiscFeature_4,SaleType_8,SaleType_6,SaleType_0,SaleType_3,SaleType_5,SaleType_4,SaleType_1,SaleType_7,SaleType_2,SaleCondition_4,SaleCondition_5,SaleCondition_0,SaleCondition_3,SaleCondition_2,SaleCondition_1
135,136,20,80.0,10400,7,6,1970,1970,288.0,0,0,1304,1304,1682,0,0,1682,0,0,2,0,3,1,7,1,1970.0,2,530,98,0,0,0,0,0,0,5,2008,0.0

__Slice out only one hot encoded features__

In [13]:
onehot_housing_X_train_1 = housing_X_train_1.loc[:,'MSZoning_3':]
onehot_housing_X_test_1 = housing_X_test_1.loc[:,'MSZoning_3':]

non_onehot_housing_X_train_1 = housing_X_train_1.loc[:,:'YrSold']
non_onehot_housing_X_test_1 = housing_X_test_1.loc[:,:'YrSold']

print(housing_X_train_1.shape)
print(onehot_housing_X_train_1.shape)
print(non_onehot_housing_X_train_1.shape)

print(housing_X_test_1.shape)
print(onehot_housing_X_test_1.shape)
print(non_onehot_housing_X_test_1.shape)

(1022, 305)
(1022, 268)
(1022, 37)
(438, 305)
(438, 268)
(438, 37)


The number of features are very high now, there are two ways to handle it next:-
1. __Dimention reduction__- Reduce number of features:
    - Principal component analysis (PCA)
    - Truncated singular value decomposition (TSVD)
2. Implement an algorithm, that can handle sparse dataset.

##### Dimention reduction : Implement PCA 

In [14]:
#Reduce the dimention of the encoded variable
num_encoded_variable = len(columns)
pca = PCA(n_components=num_encoded_variable)
onehot_housing_X_train_1_pca = pca.fit(onehot_housing_X_train_1).transform(onehot_housing_X_train_1)
print(onehot_housing_X_train_1_pca.shape)


(1022, 43)


In [15]:
#Append back to the original train dataset.
onehot_housing_X_train_1_pca_temp = pd.DataFrame(onehot_housing_X_train_1_pca,columns=[("ohe_"+str(i)) for i in range(num_encoded_variable)])
onehot_housing_X_train_1_pca_temp=onehot_housing_X_train_1_pca_temp.set_index(non_onehot_housing_X_train_1.index.values)
onehot_housing_X_train_pca = pd.concat([non_onehot_housing_X_train_1,onehot_housing_X_train_1_pca_temp],axis=1)
print(onehot_housing_X_train_pca.shape)


(1022, 80)


##### Dimention reduction : Implement TSVD

In [16]:
from sklearn.decomposition import TruncatedSVD
# Create a TSVD
tsvd = TruncatedSVD(n_components=num_encoded_variable)
# Conduct TSVD on sparse matrix
onehot_housing_X_train_1_tsvd = tsvd.fit(onehot_housing_X_train_1).transform(onehot_housing_X_train_1)
print(onehot_housing_X_train_1_tsvd.shape)

(1022, 43)


In [17]:
#Append back to the original train dataset.
onehot_housing_X_train_1_tsvd_temp = pd.DataFrame(onehot_housing_X_train_1_tsvd,columns=[("ohe_"+str(i)) for i in range(num_encoded_variable)])
onehot_housing_X_train_1_tsvd_temp=onehot_housing_X_train_1_tsvd_temp.set_index(non_onehot_housing_X_train_1.index.values)
onehot_housing_X_train_tsvd = pd.concat([non_onehot_housing_X_train_1,onehot_housing_X_train_1_tsvd_temp],axis=1)
print(onehot_housing_X_train_tsvd.shape)


(1022, 80)


#### Implement using pandas.get_dummies

In [18]:
#Divide the train & test dataset into dependent & independent variable
housing_X_train = train_set.iloc[:,:-1]
housing_y_train = train_set.iloc[:,-1:]

housing_X_test = test_set.iloc[:,:-1]
housing_y_test = test_set.iloc[:,-1:]


In [19]:
pd.get_dummies(housing_X_train)

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageFinish_Fin,GarageFinish_RFn,GarageFinish_Unf,GarageQual_Ex,GarageQual_Fa,GarageQual_Gd,GarageQual_Po,GarageQual_TA,GarageCond_Ex,GarageCond_Fa,GarageCond_Gd,GarageCond_Po,GarageCond_TA,Pav

__The same Ddimention reduction techniques mentioned above can be implemented.__

## Selection of significant features

__Some Techniques :- __
1. Univariate Selection
    -  The scikit-learn library provides the SelectKBest class that can be used with a suite of different statistical tests to select a specific number of features.
2. Recursive Feature Elimination
    -  The Recursive Feature Elimination (or RFE) works by recursively removing attributes and building a model on those attributes that remain.It uses the model accuracy to identify which attributes (and combination of attributes) contribute the most to predicting the target attribute.
3. Principal Component Analysis
    -  Principal Component Analysis (or PCA) uses linear algebra to transform the dataset into a compressed form.Generally this is called a data reduction technique. A property of PCA is that you can choose the number of dimensions or principal component in the transformed result.
4. Feature Importance
    -  Bagged decision trees like Random Forest and Extra Trees can be used to estimate the importance of features.

In [20]:
#Univariate Selection

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

X = onehot_housing_X_train_pca
Y = housing_y_train

# feature extraction
selection = SelectKBest(score_func=f_regression, k=20)
fit_obj = selection.fit(X, Y.values.ravel())
# summarize scores
print(fit_obj.scores_)
features = fit_obj.transform(X)

# summarize selected features
print(features[0:5,:])

#Get most significant features
mask = fit_obj.get_support()
new_features = onehot_housing_X_train_pca.columns[mask]
print(new_features)

[1.23761872e+00 9.11233812e+00 4.49928567e+01 7.57308705e+01
 1.63476659e+03 5.23551543e+00 3.62779082e+02 3.62747346e+02
 2.63721721e+02 1.52417812e+02 9.52837720e-02 5.35398400e+01
 5.44706440e+02 5.25554087e+02 1.12282249e+02 4.93954460e-03
 9.23036559e+02 5.61026206e+01 2.85222773e+00 4.40440687e+02
 8.60253365e+01 2.67565002e+01 2.25190875e+01 3.77296978e+02
 2.75772431e+02 8.03150154e+01 7.17793466e+02 6.43420696e+02
 1.25630752e+02 8.51305365e+01 2.26854782e+01 3.61369638e+00
 1.48961548e+01 1.98968822e+01 2.78606689e-01 2.40431194e+00
 1.66123335e-01 8.03159888e+02 2.30787521e+01 6.92667845e+00
 8.07449782e+01 5.80239822e+01 3.29812768e+00 2.45578264e+01
 1.30977112e+00 6.94829912e+00 7.14554885e-01 4.12218924e+00
 5.12545127e+00 1.53286807e+00 1.00480752e-01 4.90682198e+00
 4.22700225e-01 4.55137770e+00 3.94287559e-01 1.51100679e+00
 2.70805176e-02 1.02817771e+00 3.43922785e+00 4.74344821e+00
 2.77430427e-02 1.52443606e-01 2.06053853e+00 4.42362857e-01
 4.01434071e+00 6.936816

In [21]:
#Recursive Feature Elimination (RFE)

from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

X = onehot_housing_X_train_pca
Y = housing_y_train

model = LogisticRegression()
rfe = RFE(model, 20)
fit = rfe.fit(X, Y.values.ravel())

print("Num Features: %d") % fit.n_features_
#Get most significant features
mask = fit.support_
new_features = onehot_housing_X_train_pca.columns[mask]
print("Selected Features: %s") % new_features
print("Feature Ranking: %s") % fit.ranking_


KeyboardInterrupt: 

Above are two techniques that can be used to select most significant features.

Most significant features can also be visualized as below, using Corelation heatmap.

In [ ]:
#Compute Correlation
X = onehot_housing_X_train_pca
Y = housing_y_train

#concat both features and corresponding target variable
housing_price = pd.concat([X,Y],axis=1)

corr = housing_price.corr()

fig, ax1 = plt.subplots(nrows=1, ncols=1)
fig.set_size_inches(35,22)

img = ax1.imshow(corr, cmap='afmhot', interpolation='none', aspect='auto')
plt.colorbar(img)
plt.xticks(range(len(corr)), corr.columns, rotation='vertical')
plt.yticks(range(len(corr)), corr.columns);
plt.suptitle('House Price Correlations Heat Map', fontsize=15, fontweight='bold')
plt.show()

As Visisble from the Correlation Heat map, the following features are most significant.
1. OverallQual
2. YearBuilt
3. YearRemodAdded
4. TotalBsmtSF
5. 1stFlrSF
6. GrLivArea
7. FullBath
8. TotRmsAbvGrd
9. GarageCars
10. GarageArea
11. ohe_0

# Inference
1. Feature Selection helps reduce training time & model complexity.
2. Feature selection requires, the data to be in numeric format.
3. There are multiple ways to convert string categorical data to numeric/binary as below:-
    - Integer Encoding
    - One-Hot Encoding
    - Frequency-based encoding
    - Target mean encoding
    - Binary encoding
    - Hash encoding
4. One-Hot Encoding converts the categorical data into binary representation, but requires the categorical data to be in numeric format.
5. To convert string categorical data to numeric format, we use Label encoding.
6. If 
7. pandas.get_dummies(), can also be used to convert categorical to binary representation. Though pandas.get_dummies() cannot be added as part of a pipeline.
8. Using One-Hot Encoding or pandas.get_dummies(), the number of features increase significantly, below dimention reduction techniques can be used to reduce dimentions:-
    - PCA
    - TSVD
9. Once the data is convered into numberic/binary representation & dimention reduction is done,we can implement one of the below Feature selection process :-
    - Univariate Selection
    - Recursive Feature Elimination
    - Visualization of correlation Heatmap
10. Feature hashing can be used for test processing.

# Reference
1. https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/
2. https://dzone.com/articles/handling-character-data-for-machine-learning
3. https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/
4. https://stackoverflow.com/questions/47460387/how-do-i-resolve-one-hot-encoding-if-my-test-data-has-missing-values-in-a-col
5. https://www.analyticsvidhya.com/blog/2017/06/a-comprehensive-guide-for-linear-ridge-and-lasso-regression/
6. https://machinelearningmastery.com/feature-selection-machine-learning-python/